<!-- https://www.kaggle.com/code/iakhtar0/63-next-word-predictor-lstm-campusx -->

In [1]:
import sys
import platform
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import nltk
import sklearn

print("🔧 Environment Information")
print("=" * 40)
print(f"Python version      : {platform.python_version()}")
print(f"TensorFlow version  : {tf.__version__}")
print(f"NumPy version       : {np.__version__}")
print(f"Pandas version      : {pd.__version__}")
print(f"Matplotlib version  : {matplotlib.__version__}")
print(f"Seaborn version     : {sns.__version__}") # type: ignore
print(f"NLTK version        : {nltk.__version__}")
print(f"scikit-learn version: {sklearn.__version__}")
print("=" * 40)

# Optional: GPU info
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✅ GPU detected: {gpus[0].name}")
else:
    print("⚠️ No GPU detected (running on CPU).")
tf.keras.backend.clear_session()
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

2025-12-29 19:24:16.260899: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/srirama/sr_proj/tf-venv/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


🔧 Environment Information
Python version      : 3.12.3
TensorFlow version  : 2.20.0
NumPy version       : 2.4.0
Pandas version      : 2.3.3
Matplotlib version  : 3.10.8
Seaborn version     : 0.13.2
NLTK version        : 3.9.2
scikit-learn version: 1.8.0
✅ GPU detected: /physical_device:GPU:0


In [2]:
# Deep Learning library
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras import backend as K
print(tf.__version__)

2.20.0


In [3]:
from pathlib import Path
NOTEBOOK_DIR  = Path().resolve()
BASE_DIR = NOTEBOOK_DIR.parents[1]
DATASET_DIR = BASE_DIR / "src" / "artifacts" /  "data" / 'Twitter' / "preprocessed"
DATASET_FILE_PATH = DATASET_DIR  / 'train.csv'
print(DATASET_FILE_PATH.exists())
DATASET_FILE_PATH


True


PosixPath('/home/srirama/sr_proj/EmotionAnalysis/src/artifacts/data/Twitter/preprocessed/train.csv')

In [4]:
df = pd.read_csv(DATASET_FILE_PATH,usecols=["sentiment","cleaned_text"],nrows=1000)
df["cleaned_text"] = df["cleaned_text"].astype(str)
df.isnull().sum()

sentiment       0
cleaned_text    0
dtype: int64

In [5]:
# Create a CountVectorizer Object
vectorizer = tf.keras.layers.TextVectorization(
    output_mode="tf_idf",
    # output_mode="int",
    ngrams=None,
)
corpus  = df["cleaned_text"]
vectorizer.adapt(corpus)
tfidf_matrix = vectorizer(corpus)
vocab_size = len(vectorizer.get_vocabulary())
print("Size of vocab : ",vocab_size)

I0000 00:00:1767036290.055254    5706 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1767 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2050, pci bus id: 0000:01:00.0, compute capability: 8.6


Size of vocab :  1732


In [6]:
import rich
rich.print(tfidf_matrix)
print("Shape of matric",tfidf_matrix.shape)

tf.Tensor(
[[0.        1.0360919 1.3736782 ... 0.        0.        0.       ]
 [0.        0.        1.3736782 ... 0.        0.        0.       ]
 [0.        1.0360919 1.3736782 ... 0.        0.        0.       ]
 ...
 [0.        0.        1.3736782 ... 0.        0.        0.       ]
 [0.        0.        1.3736782 ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]], shape=(1000, 1732), dtype=float32)

Shape of matric (1000, 1732)


In [7]:
# your labels (strings)
labels = df['sentiment']
# 1. Create lookup layer
lookup = tf.keras.layers.StringLookup(output_mode="int", num_oov_indices=0)
lookup.adapt(labels)
# 2. Convert labels to integer IDs
y_int = lookup(labels)
# 3. Convert to one-hot
num_classes = lookup.vocabulary_size()
y_onehot = tf.one_hot(y_int, depth=num_classes)

print("First 5 Int labels:", y_int[:5])
print("First 5  One-hot:", y_onehot[:5])
print("Vocabulary:", lookup.get_vocabulary())
print("Shape of y_onehot",y_onehot.shape)

First 5 Int labels: tf.Tensor([0 0 0 0 0], shape=(5,), dtype=int64)
First 5  One-hot: tf.Tensor(
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]], shape=(5, 4), dtype=float32)
Vocabulary: [np.str_('Positive'), np.str_('Neutral'), np.str_('Negative'), np.str_('Irrelevant')]
Shape of y_onehot (1000, 4)


In [8]:
from keras.models import Sequential
from keras.layers import SimpleRNN , Dense , Embedding 
import mlflow
import mlflow.tensorflow
import warnings
warnings.filterwarnings("ignore")
print("Project base dir is :",BASE_DIR)

Project base dir is : /home/srirama/sr_proj/EmotionAnalysis


In [9]:
tracking_path = f"http://localhost:5000/"
mlflow.set_tracking_uri(tracking_path)
print("Tracking URI:", mlflow.get_tracking_uri())
mlflow.set_experiment("1-KERAS-TUNER")

2025/12/29 19:25:18 INFO mlflow.tracking.fluent: Experiment with name '1-KERAS-TUNER' does not exist. Creating a new experiment.


Tracking URI: http://localhost:5000/


<Experiment: artifact_location='mlflow-artifacts:/5', creation_time=1767036318507, experiment_id='5', last_update_time=1767036318507, lifecycle_stage='active', name='1-KERAS-TUNER', tags={}>

In [10]:
tfidf_matrix.shape , y_onehot.shape

(TensorShape([1000, 1732]), TensorShape([1000, 4]))

In [11]:
import tensorflow as tf
# number of samples
n = tfidf_matrix.shape[0]
idx = tf.random.shuffle(tf.range(n))
X = tf.gather(tfidf_matrix, idx)
y = tf.gather(y_onehot, idx)

split = int(0.6 * n)

X_train, X_val = X[:split], X[split:]
y_train, y_val = y[:split], y[split:]


In [12]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_ds   = tf.data.Dataset.from_tensor_slices((X_val, y_val))

BATCH_SIZE = 32

train_ds = (
    train_ds
    .shuffle(buffer_size=len(X_train))
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

val_ds = (
    val_ds
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)


In [13]:
import keras_tuner as kt
from tensorflow.keras.callbacks import ReduceLROnPlateau
# Define constants
timestep = X.shape[1]
vocab_size = X.shape[1] #
embed_dim = 16                 # small, efficient
rnn_units = 32  
num_classes = y.shape[1]

In [14]:
class CustomHyperModel(kt.HyperModel):
    TRIAL_ID : int = 0
    def build(self,hp):
        model = Sequential()
        # Bigger embedding
        model.add(Embedding(
            input_dim=vocab_size,
            output_dim=hp.Choice('embed_dim_units', [32, 64, 128]),
            input_length=timestep
        ))
        # Stacked SimpleRNN — way more expressive
        model.add(SimpleRNN(
            hp.Choice('rnn_units_1', [32, 64, 128]),
            return_sequences=True,
            dropout=0.2
        ))
        model.add(SimpleRNN(
            hp.Choice('rnn_units_2', [16, 32, 64]),
            return_sequences=False,
            dropout=0.2
        ))
        # Dense layers as before, but with more meaningful size
        model.add(Dense(hp.Int('dense_1_units', 32, 128, step=16), activation='relu'))
        model.add(Dense(hp.Int('dense_2_units', 16, 64, step=16), activation='relu'))
        model.add(Dense(num_classes, activation='softmax'))
        model.compile(
            loss='categorical_crossentropy',
            optimizer="adam",
            metrics=['accuracy']
        )
        return model
    def fit(self, hp, model, *args, **kwargs):
        CustomHyperModel.TRIAL_ID+=1
        with mlflow.start_run(run_name=f"trial_{CustomHyperModel.TRIAL_ID}"):
            mlflow.log_params(hp.values)
            mlflow.tensorflow.autolog()
            return model.fit(*args , **kwargs)
        

In [15]:
tuner = kt.Hyperband(
    hypermodel = CustomHyperModel(),
    objective = 'val_loss',
    max_epochs = 20,
    overwrite=True,
    directory="/tmp/tb",
)

In [ ]:
tuner.search(
    train_ds, validation_data=val_ds,
    epochs = 10, 
    # callbacks = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
)

Trial 5 Complete [00h 02m 27s]
val_loss: 1.295479416847229

Best val_loss So Far: 1.2906771898269653
Total elapsed time: 00h 12m 21s

Search: Running Trial #6

Value             |Best Value So Far |Hyperparameter
128               |64                |embed_dim_units
32                |64                |rnn_units_1
32                |16                |rnn_units_2
96                |64                |dense_1_units
16                |16                |dense_2_units
3                 |3                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round



2025/12/29 19:38:02 WARNING mlflow.tensorflow: Encountered unexpected error while inferring batch size from training dataset: Sequential model 'sequential' has no defined input shape yet.
2025-12-29 19:38:02.547237: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1/3
 8/19 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.2202 - loss: 1.4100

18/19 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.3004 - loss: 1.3734

In [49]:
tuner.results_summary()

Results summary
Results in /tmp/tb/untitled_project
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 0016 summary
Hyperparameters:
units: 32
lr: 0.0001
dense_1_units: 64
dense_2_units: 40
dense_3_units: 24
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0012
Score: 1.3497672080993652

Trial 0017 summary
Hyperparameters:
units: 32
lr: 0.0001
dense_1_units: 40
dense_2_units: 56
dense_3_units: 64
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0013
Score: 1.3538638353347778

Trial 0012 summary
Hyperparameters:
units: 32
lr: 0.0001
dense_1_units: 64
dense_2_units: 40
dense_3_units: 24
tuner/epochs: 7
tuner/initial_epoch: 3
tuner/bracket: 2
tuner/round: 1
tuner/trial_id: 0008
Score: 1.3591728210449219

Trial 0013 summary
Hyperparameters:
units: 32
lr: 0.0001
dense_1_units: 40
dense_2_units: 56
dense_3_units: 64
tuner/epochs: 7
tuner/initial_epoch: 3
tuner/bracket: 2
tuner/round: 1
tuner/

In [50]:
best_model = tuner.get_best_models()[0]
best_hyperparameters= tuner.get_best_hyperparameters()[0].values
best_hyperparameters

{'units': 32,
 'lr': 0.0001,
 'dense_1_units': 64,
 'dense_2_units': 40,
 'dense_3_units': 24,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'tuner/trial_id': '0012'}

In [52]:
best_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 32)             │         1,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 40)             │         2,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 24)             │           984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           100 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,884 (26.89 KB)

 Trainable params: 6,884 (26.89 KB)

 Non-trainable params: 0 (0.00 B)

In [53]:
best_model.save("best_model.keras")

In [54]:
model = tf.keras.models.load_model("best_model.keras")

In [56]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 32)             │         1,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 40)             │         2,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 24)             │           984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           100 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,654 (80.68 KB)

 Trainable params: 6,884 (26.89 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 13,770 (53.79 KB)